In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [3]:
year = 2018
exclude = '_no_Others'

In [4]:
df = pd.read_csv('data/news_predictions/news_{}_predictions.csv'.format(year))
df = df.drop(['Unnamed: 0'], axis=1)
df  = df.dropna(subset=['transcript'])
df = df.reset_index(drop=True)

FileNotFoundError: [Errno 2] File data/news_predictions/news_2018_predictions.csv does not exist: 'data/news_predictions/news_2018_predictions.csv'

In [36]:
df

,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
0,163795,Belfast Telegraph,1,1,2015,Plans for site of former factory to be 12-acre...,"The Artnz Belting company,\nwhich had been a m...",1,Transport,48.68,"Parliament, government and politics",12.64,Communities and families,12.03
1,163795,Belfast Telegraph,1,1,2015,Councillors left red-faced over bid for new pr...,"However, it was then revealed that a database ...",1,Communities and families,28.51,Transport,15.06,Others,10.18
2,163795,Belfast Telegraph,1,1,2015,Woman breached bail and spent Christmas in cus...,Sharon Campbell from Lawrence Hill in Londonde...,1,Others,25.44,"Crime, civil law, justice and rights",24.46,"Asylum, immigration and nationality",12.85
3,163795,Belfast Telegraph,1,1,2015,Driver who was three times legal limit jailed,Paul McCloskey (36) of Horace Street in London...,1,"Crime, civil law, justice and rights",35.05,Others,10.45,Defence,8.50
4,163795,Belfast Telegraph,1,1,2015,A oneness that unites us in Christ;\nMY VIEW,That means more than simply recording the numb...,1,"Parliament, government and politics",53.14,International affairs,12.30,"Culture, media and sport",12.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619553,412338,Wales,31,12,2015,Morning news headlines: Homes evacuated and th...,Hundreds of homes have been evacuated and thou...,1,Others,30.94,"Business, industry and consumers",17.66,"Parliament, government and politics",13.39
1619554,412338,Wales,31,12,2015,How many parents have been fined for taking th...,Figures gathered by WalesOnline reveal a postc...,1,Education,36.93,"Parliament, government and politics",25.48,Communities and families,18.26
1619555,412338,Wales,31,12,2015,Enterprise zone based firms across Wales get £...,"Economy Minister, Edwina Hart announced the aw...",1,"Parliament, government and politics",80.30,"Business, industry and consumers",4.78,"Culture, media and sport",3.82
1619556,412338,Wales,31,12,2015,Changes to Wales' organ donation system will s...,Carwyn Jones was among the politicians to deli...,1,"Parliament, government and politics",90.38,Communities and families,1.62,"Culture, media and sport",1.33


In [37]:
df = df.sample(10000)
df

,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
479636,411938,Standard,26,4,2015,"Troy Deeney and Alex Pritchard star, but Chels...","However Patrick Bamford, the Chelsea loanee wh...",1,"Culture, media and sport",29.34,"Business, industry and consumers",12.07,"Parliament, government and politics",9.41
70996,389195,Telegraph,19,1,2015,Jose Mourinho: Chelsea fans should stop taunti...,The Chelsea manger also reassured Liverpool fa...,1,Others,34.72,"Parliament, government and politics",14.73,"Culture, media and sport",12.74
653095,408506,Express,6,6,2015,Lord Griffiths. Law lord and president of the ...,PHOTOSHOT Lord Griffiths enjoyed a great legal...,1,"Crime, civil law, justice and rights",27.69,Defence,18.20,Others,17.68
602901,138794,Mail,25,5,2015,SHOULD I HIDE MY RACY PAST?,BY ROWAN PELLING QUESTION: When I met my other...,1,"Crime, civil law, justice and rights",37.47,International affairs,26.50,Others,6.43
413544,8200,Independent,11,4,2015,London arts 'visionary' dies in cycle accident,"Ms Gemmill, 55, had recently been appointed as...",1,"Parliament, government and politics",25.16,Transport,17.08,"Culture, media and sport",15.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823197,408506,Express,16,7,2015,Heathrow holiday chaos as check-in machines 'r...,STEVE REIGATE Passengers faced the misery of l...,1,Transport,66.89,European Union,10.94,International affairs,6.16
754372,138794,Mail,30,6,2015,DIARY,But then there's the rise of the Christian rig...,1,"Culture, media and sport",22.97,"Crime, civil law, justice and rights",20.94,Others,19.64
464795,8200,Independent,23,4,2015,Whitney Museum of American Art opens the doors...,There had been intense interest and buzz in th...,1,"Culture, media and sport",45.00,Others,34.50,"Business, industry and consumers",5.27
50147,397135,Mail,14,1,2015,Sharon Stone shows off her lean physique in sk...,The pair were seen leaving Mr. Chow restaurant...,1,Others,26.41,"Agriculture, animals, food and rural affairs",14.72,"Culture, media and sport",10.24


In [38]:
from joblib import dump, load

enc = load('./models/doc2vec/encoder_{}_{}{}.joblib'.format(2009, 2013, exclude))

def predict(X, topn=3):
    preprocessed_X = preprocess_text(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = classifier.predict(vec_X)
    pred_i = [(topics_index_to_name_map[i], p) for i,p in enumerate(pred[0])]
    pred_sorted = sorted(pred_i, key=lambda x: x[1], reverse=True)
    return pred_sorted[:topn]


def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(len(enc.categories_[0]), activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [39]:
doc2vec_model = Doc2Vec.load('./models/doc2vec/doc2vec_{}_{}{}'.format(2009, 2013, exclude))
classifier = build_network()
classifier.load_weights('./models/doc2vec/classifier_{}_{}{}'.format(2009, 2013, exclude))

In [40]:
transcripts = df.transcript.values
predict(transcripts[5])

[('Economy and finance', 0.7643188),
 ('Business, industry and consumers', 0.15269469),
 ('Transport', 0.016088912)]

In [41]:
predictions = []
from tqdm import tqdm
for transcript in tqdm(transcripts):
    predictions.append(predict(transcript))





  0%|          | 0/10000 [00:00<?, ?it/s]



  0%|          | 2/10000 [00:00<16:47,  9.92it/s]



  0%|          | 4/10000 [00:00<15:36, 10.67it/s]



  0%|          | 6/10000 [00:00<14:10, 11.74it/s]



  0%|          | 7/10000 [00:00<22:01,  7.56it/s]



  0%|          | 8/10000 [00:00<22:13,  7.50it/s]



  0%|          | 9/10000 [00:01<24:13,  6.87it/s]



  0%|          | 11/10000 [00:01<27:03,  6.15it/s]



  0%|          | 13/10000 [00:01<23:09,  7.19it/s]



  0%|          | 15/10000 [00:01<20:18,  8.20it/s]



  0%|          | 17/10000 [00:01<19:36,  8.49it/s]



  0%|          | 19/10000 [00:02<17:42,  9.40it/s]



  0%|          | 21/10000 [00:02<18:49,  8.84it/s]



  0%|          | 23/10000 [00:02<16:43,  9.94it/s]



  0%|          | 25/10000 [00:02<17:32,  9.48it/s]



  0%|          | 27/10000 [00:02<15:41, 10.59it/s]



  0%|          | 29/10000 [00:03<17:08,  9.70it/s]



  0%|          | 31/10000 [00:03<17:07,  9.71it/s]



  0%|          | 33/10000 [00:03<14:40,

  5%|▌         | 513/10000 [00:40<12:48, 12.35it/s]



  5%|▌         | 515/10000 [00:40<12:56, 12.21it/s]



  5%|▌         | 517/10000 [00:41<12:29, 12.65it/s]



  5%|▌         | 519/10000 [00:41<12:34, 12.56it/s]



  5%|▌         | 521/10000 [00:41<12:00, 13.15it/s]



  5%|▌         | 523/10000 [00:41<11:57, 13.21it/s]



  5%|▌         | 525/10000 [00:41<12:05, 13.05it/s]



  5%|▌         | 527/10000 [00:41<12:06, 13.03it/s]



  5%|▌         | 529/10000 [00:42<11:43, 13.47it/s]



  5%|▌         | 531/10000 [00:42<10:42, 14.73it/s]



  5%|▌         | 533/10000 [00:42<11:24, 13.83it/s]



  5%|▌         | 535/10000 [00:42<11:44, 13.43it/s]



  5%|▌         | 537/10000 [00:42<11:12, 14.08it/s]



  5%|▌         | 539/10000 [00:42<11:49, 13.33it/s]



  5%|▌         | 541/10000 [00:42<12:28, 12.63it/s]



  5%|▌         | 543/10000 [00:43<13:08, 11.99it/s]



  5%|▌         | 545/10000 [00:43<13:14, 11.91it/s]



  5%|▌         | 547/10000 [00:43<12:57, 12.16it/s]



  5%|▌    

 10%|█         | 1036/10000 [01:21<12:41, 11.77it/s]



 10%|█         | 1038/10000 [01:21<12:20, 12.10it/s]



 10%|█         | 1040/10000 [01:21<11:49, 12.64it/s]



 10%|█         | 1042/10000 [01:21<13:17, 11.24it/s]



 10%|█         | 1044/10000 [01:22<13:44, 10.87it/s]



 10%|█         | 1046/10000 [01:22<14:24, 10.36it/s]



 10%|█         | 1048/10000 [01:22<12:45, 11.69it/s]



 10%|█         | 1050/10000 [01:22<12:31, 11.91it/s]



 11%|█         | 1052/10000 [01:22<11:55, 12.50it/s]



 11%|█         | 1054/10000 [01:22<11:57, 12.46it/s]



 11%|█         | 1056/10000 [01:23<13:22, 11.14it/s]



 11%|█         | 1058/10000 [01:23<12:29, 11.93it/s]



 11%|█         | 1061/10000 [01:23<10:51, 13.73it/s]



 11%|█         | 1064/10000 [01:23<09:43, 15.31it/s]



 11%|█         | 1066/10000 [01:23<11:59, 12.41it/s]



 11%|█         | 1068/10000 [01:23<11:43, 12.70it/s]



 11%|█         | 1070/10000 [01:24<10:44, 13.85it/s]



 11%|█         | 1072/10000 [01:24<11:29, 12.95i

 15%|█▌        | 1535/10000 [02:01<10:10, 13.87it/s]



 15%|█▌        | 1537/10000 [02:01<09:32, 14.78it/s]



 15%|█▌        | 1539/10000 [02:01<09:45, 14.45it/s]



 15%|█▌        | 1541/10000 [02:01<10:37, 13.27it/s]



 15%|█▌        | 1543/10000 [02:01<10:23, 13.57it/s]



 15%|█▌        | 1545/10000 [02:01<09:54, 14.23it/s]



 15%|█▌        | 1548/10000 [02:01<08:52, 15.86it/s]



 16%|█▌        | 1550/10000 [02:01<09:22, 15.01it/s]



 16%|█▌        | 1552/10000 [02:02<08:53, 15.83it/s]



 16%|█▌        | 1554/10000 [02:02<08:46, 16.03it/s]



 16%|█▌        | 1556/10000 [02:02<09:55, 14.17it/s]



 16%|█▌        | 1558/10000 [02:02<10:41, 13.16it/s]



 16%|█▌        | 1560/10000 [02:02<10:10, 13.83it/s]



 16%|█▌        | 1562/10000 [02:02<12:32, 11.22it/s]



 16%|█▌        | 1564/10000 [02:03<13:52, 10.14it/s]



 16%|█▌        | 1566/10000 [02:03<13:42, 10.26it/s]



 16%|█▌        | 1568/10000 [02:03<13:17, 10.57it/s]



 16%|█▌        | 1570/10000 [02:03<13:05, 10.73i

 20%|██        | 2034/10000 [02:42<10:48, 12.29it/s]



 20%|██        | 2036/10000 [02:43<12:09, 10.91it/s]



 20%|██        | 2038/10000 [02:43<11:04, 11.98it/s]



 20%|██        | 2040/10000 [02:43<11:23, 11.64it/s]



 20%|██        | 2042/10000 [02:43<12:50, 10.32it/s]



 20%|██        | 2044/10000 [02:43<13:27,  9.85it/s]



 20%|██        | 2046/10000 [02:44<13:05, 10.13it/s]



 20%|██        | 2048/10000 [02:44<13:14, 10.01it/s]



 20%|██        | 2050/10000 [02:44<12:07, 10.93it/s]



 21%|██        | 2052/10000 [02:44<11:12, 11.82it/s]



 21%|██        | 2054/10000 [02:44<11:39, 11.35it/s]



 21%|██        | 2056/10000 [02:44<11:24, 11.61it/s]



 21%|██        | 2058/10000 [02:45<11:14, 11.77it/s]



 21%|██        | 2060/10000 [02:45<10:22, 12.75it/s]



 21%|██        | 2062/10000 [02:45<09:54, 13.36it/s]



 21%|██        | 2064/10000 [02:45<10:16, 12.87it/s]



 21%|██        | 2066/10000 [02:45<09:24, 14.05it/s]



 21%|██        | 2068/10000 [02:45<08:43, 15.16i

...

 76%|███████▌  | 7609/10000 [09:52<03:20, 11.93it/s]



 76%|███████▌  | 7611/10000 [09:52<02:57, 13.46it/s]



 76%|███████▌  | 7613/10000 [09:52<02:50, 14.02it/s]



 76%|███████▌  | 7615/10000 [09:52<02:36, 15.25it/s]



 76%|███████▌  | 7617/10000 [09:52<02:35, 15.33it/s]



 76%|███████▌  | 7619/10000 [09:52<02:39, 14.96it/s]



 76%|███████▌  | 7622/10000 [09:53<02:26, 16.24it/s]



 76%|███████▌  | 7624/10000 [09:53<02:25, 16.34it/s]



 76%|███████▋  | 7626/10000 [09:53<02:25, 16.31it/s]



 76%|███████▋  | 7628/10000 [09:53<02:38, 14.96it/s]



 76%|███████▋  | 7630/10000 [09:53<02:33, 15.43it/s]



 76%|███████▋  | 7632/10000 [09:53<02:24, 16.42it/s]



 76%|███████▋  | 7634/10000 [09:53<02:31, 15.63it/s]



 76%|███████▋  | 7636/10000 [09:54<03:03, 12.89it/s]



 76%|███████▋  | 7639/10000 [09:54<02:40, 14.71it/s]



 76%|███████▋  | 7641/10000 [09:54<02:32, 15.48it/s]



 76%|███████▋  | 7643/10000 [09:54<02:35, 15.20it/s]



 76%|███████▋  | 7645/10000 [09:54<02:30, 15.61i

 81%|████████  | 8115/10000 [10:33<02:59, 10.51it/s]



 81%|████████  | 8117/10000 [10:34<02:59, 10.46it/s]



 81%|████████  | 8119/10000 [10:34<02:48, 11.17it/s]



 81%|████████  | 8121/10000 [10:34<02:47, 11.23it/s]



 81%|████████  | 8123/10000 [10:34<02:57, 10.55it/s]



 81%|████████▏ | 8125/10000 [10:34<03:04, 10.15it/s]



 81%|████████▏ | 8127/10000 [10:35<02:56, 10.61it/s]



 81%|████████▏ | 8129/10000 [10:35<02:54, 10.75it/s]



 81%|████████▏ | 8131/10000 [10:35<02:54, 10.72it/s]



 81%|████████▏ | 8133/10000 [10:35<02:43, 11.44it/s]



 81%|████████▏ | 8135/10000 [10:35<02:50, 10.95it/s]



 81%|████████▏ | 8137/10000 [10:36<03:19,  9.33it/s]



 81%|████████▏ | 8138/10000 [10:36<03:48,  8.13it/s]



 81%|████████▏ | 8139/10000 [10:36<04:17,  7.23it/s]



 81%|████████▏ | 8140/10000 [10:36<03:59,  7.75it/s]



 81%|████████▏ | 8141/10000 [10:36<03:55,  7.88it/s]



 81%|████████▏ | 8143/10000 [10:36<03:16,  9.45it/s]



 81%|████████▏ | 8145/10000 [10:36<03:12,  9.65i

 86%|████████▌ | 8600/10000 [11:13<02:08, 10.88it/s]



 86%|████████▌ | 8602/10000 [11:13<01:51, 12.53it/s]



 86%|████████▌ | 8604/10000 [11:13<01:43, 13.47it/s]



 86%|████████▌ | 8606/10000 [11:13<01:35, 14.59it/s]



 86%|████████▌ | 8608/10000 [11:13<01:52, 12.37it/s]



 86%|████████▌ | 8611/10000 [11:14<01:40, 13.78it/s]



 86%|████████▌ | 8613/10000 [11:14<01:37, 14.17it/s]



 86%|████████▌ | 8615/10000 [11:14<01:37, 14.17it/s]



 86%|████████▌ | 8617/10000 [11:14<01:37, 14.23it/s]



 86%|████████▌ | 8619/10000 [11:14<01:29, 15.43it/s]



 86%|████████▌ | 8622/10000 [11:14<01:22, 16.76it/s]



 86%|████████▌ | 8624/10000 [11:14<01:20, 17.03it/s]



 86%|████████▋ | 8626/10000 [11:14<01:19, 17.22it/s]



 86%|████████▋ | 8628/10000 [11:15<01:29, 15.34it/s]



 86%|████████▋ | 8630/10000 [11:15<01:23, 16.49it/s]



 86%|████████▋ | 8632/10000 [11:15<01:22, 16.57it/s]



 86%|████████▋ | 8634/10000 [11:15<01:26, 15.81it/s]



 86%|████████▋ | 8636/10000 [11:15<01:27, 15.51i

 91%|█████████ | 9100/10000 [11:52<01:39,  9.08it/s]



 91%|█████████ | 9102/10000 [11:53<01:32,  9.72it/s]



 91%|█████████ | 9104/10000 [11:53<01:34,  9.47it/s]



 91%|█████████ | 9106/10000 [11:53<01:26, 10.38it/s]



 91%|█████████ | 9108/10000 [11:53<01:24, 10.52it/s]



 91%|█████████ | 9110/10000 [11:53<01:26, 10.32it/s]



 91%|█████████ | 9112/10000 [11:54<01:30,  9.81it/s]



 91%|█████████ | 9114/10000 [11:54<01:21, 10.91it/s]



 91%|█████████ | 9116/10000 [11:54<01:10, 12.48it/s]



 91%|█████████ | 9118/10000 [11:54<01:17, 11.32it/s]



 91%|█████████ | 9120/10000 [11:54<01:16, 11.50it/s]



 91%|█████████ | 9122/10000 [11:54<01:20, 10.97it/s]



 91%|█████████ | 9124/10000 [11:55<01:11, 12.34it/s]



 91%|█████████▏| 9126/10000 [11:55<01:12, 12.01it/s]



 91%|█████████▏| 9128/10000 [11:55<01:08, 12.66it/s]



 91%|█████████▏| 9130/10000 [11:55<01:13, 11.82it/s]



 91%|█████████▏| 9132/10000 [11:55<01:10, 12.39it/s]



 91%|█████████▏| 9134/10000 [11:55<01:04, 13.37i

 96%|█████████▋| 9629/10000 [12:32<00:37,  9.84it/s]



 96%|█████████▋| 9631/10000 [12:32<00:36, 10.01it/s]



 96%|█████████▋| 9633/10000 [12:32<00:33, 10.88it/s]



 96%|█████████▋| 9635/10000 [12:32<00:33, 10.96it/s]



 96%|█████████▋| 9637/10000 [12:32<00:33, 10.76it/s]



 96%|█████████▋| 9639/10000 [12:33<00:31, 11.53it/s]



 96%|█████████▋| 9641/10000 [12:33<00:33, 10.82it/s]



 96%|█████████▋| 9643/10000 [12:33<00:28, 12.46it/s]



 96%|█████████▋| 9645/10000 [12:33<00:28, 12.60it/s]



 96%|█████████▋| 9647/10000 [12:33<00:29, 11.99it/s]



 96%|█████████▋| 9649/10000 [12:33<00:25, 13.51it/s]



 97%|█████████▋| 9651/10000 [12:34<00:32, 10.81it/s]



 97%|█████████▋| 9653/10000 [12:34<00:28, 12.34it/s]



 97%|█████████▋| 9655/10000 [12:34<00:30, 11.35it/s]



 97%|█████████▋| 9657/10000 [12:34<00:26, 12.76it/s]



 97%|█████████▋| 9659/10000 [12:34<00:27, 12.26it/s]



 97%|█████████▋| 9661/10000 [12:34<00:25, 13.19it/s]



 97%|█████████▋| 9663/10000 [12:34<00:26, 12.93i

In [42]:
df['predictions_without_pgp'] = predictions

In [43]:
df.to_csv('news_predictions_2015_without_pgp.csv')